In [1]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [30, 63, 125, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [2]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils

SOURCE_PATH, DEST_PATH = utils.main_processor(
    data_dir='..\data',  # search project ..\data
    downloads_dir='',  # None searchs Downloads dir, '' omits search
    downloads_limit=0,  # search the first 10 files
    clean_name_override='df_perf_ratios.pkl',  # override filename
    start_file_pattern='df_OHLCV_', # search for files starting with 'df_'
)



Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\python310.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py310\\stocks\\src']


**Available 'df_OHLCV_' files:**

- (1) `[DATA]` `df_OHLCV_2025-03-07_clean.pkl` <span style='color:#00ffff'>(45.60 MB, 2025-03-09 12:53)</span>


Input a number to select file (1-1)



    **Selected paths:**
    - Source: `..\data\df_OHLCV_2025-03-07_clean.pkl`  
    - Destination: `..\data\df_perf_ratios.pkl`
    

In [3]:
import pandas as pd

# Load the DataFrame from the pickle file
df = pd.read_pickle(SOURCE_PATH)

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
AAPL   2025-03-07     239.07
       2025-03-06     235.33
       2025-03-05     235.74
       2025-03-04     235.93
       2025-03-03     238.03
...                      ...
FCOM   2024-03-14      48.10
       2024-03-13      48.22
       2024-03-12      48.20
       2024-03-11      47.87
       2024-03-08      47.87

[620750 rows x 1 columns]

In [4]:
import utils

list_dfs = utils.get_latest_dfs(df, DAYS_RATIO)

In [5]:
all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
      Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 63d  Sortino 63d  Omega 63d  \
A      -5.993620    -6.466958   0.339012   -1.674944    -2.234644   0.749575   
AA     -2.155763    -2.666636   0.684955   -2.977692    -3.484073   0.595495   
AAL    -6.161390    -6.616198   0.345950   -0.886122    -1.413585   0.834315   
AAON   -4.036187    -4.292629   0.397821   -2.681261    -2.930634   0.544580   
AAP    -3.596952    -3.990680   0.475036   -1.322611    -1.593380   0.778788   
...          ...          ...        ...         ...          ...        ...   
ZM     -1.042746    -1.420463   0.797720   -1.124288    -1.520347   0.804002   
ZS      1.786321     2.972718   1.327855    0.203157     0.288942   1.032987   
ZTO     1.951694     2.910301   1.343725    0.974938     1.404762   1.163057   
ZTS     0.313751     0.411443   1.054799   -0.449735    -0.577324   0.928823   
ZWS    -3.474744    -3.957712   0.562681   -2.711818    -3.164389   0.644772   

In [6]:
# Save the combined DataFrame to pickle file
combined_df.to_pickle(DEST_PATH)
print(f"DataFrame successfully saved to {DEST_PATH}")

DataFrame successfully saved to ..\data\df_perf_ratios.pkl


In [7]:
_symbols = ['AAPL', 'MSFT', 'GOOG', 'NVDA', 'IBIT', 'GLD', 'VCIT']

from tabulate import tabulate

# Create a DataFrame with only the selected symbols
selected_df = combined_df.loc[_symbols]

# Display the formatted table
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 63d |   Sortino 63d |   Omega 63d |   Sharpe 125d |   Sortino 125d |   Omega 125d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+==============+===============+=============+==============+===============+=============+===============+================+==============+===============+================+==============+
| AAPL |       2.3427 |        3.7633 |      1.4917 |      -0.2936 |       -0.3939 |      0.9512 |        0.6798 |         0.9495 |       1.1233 |        1.3981 |         2.1201 |       1.2752 |
+------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
| MSFT |      -3.7995 |  